In [1]:
import csv
import numpy as np
import pandas as pd
import sys
import tifffile
import zipfile
from datetime import datetime, timedelta
from scipy.interpolate import RegularGridInterpolator
import numpy as np

Grab some example data

Download bathymetry.zip from https://globalfishingwatch.org/data-download/datasets/public-bathymetry-v1. You'll need to register for a free account. I found that Safari had problems downloading from the our public data, so you may want to use a different browser. Copy the file to the untracked directory or edit the location below.

Then download fleet-daily-csvs-100-v2-2020.zip from https://globalfishingwatch.org/data-download/datasets/public-fishing-effort.

In [2]:
fleet_path = '../../untracked/fleet-daily-csvs-100-v2-2020.zip'
start_date = datetime(2020, 1, 1)
end_date = datetime(2020, 12, 31) 

# Fishing data is stored in a stack of daily CSVs, so add them up
# This takes a few minutes.
activity_raster = np.zeros([180 * 100, 360 * 100])
date = start_date
with zipfile.ZipFile(fleet_path) as archive:
    while date <= end_date:
        # Provide some feedback so we can tell something is happening
        if date.day == 1:
            if date != start_date:
                print()
            print(f'{date.month:>2d}:', end='')
        print('.', flush=True, end='')
        with archive.open(f'fleet-daily-csvs-100-v2-2020/{date:%Y-%m-%d.csv}') as f:
            df = pd.read_csv(f)
            i = ((df.cell_ll_lat.values + 90) * 100).astype(int)
            j = ((df.cell_ll_lon.values + 180) * 100).astype(int)
            activity_raster[i, j] += df.hours
        date += timedelta(days=1)



 1:...............................
 2:.............................
 3:...............................
 4:..............................
 5:...............................
 6:..............................
 7:...............................
 8:...............................
 9:..............................
10:...............................
11:..............................
12:...............................

In [3]:
bathymetry_path = '../../untracked/bathymetry.zip'
with zipfile.ZipFile(bathymetry_path) as archive:
    with archive.open('bathymetry.tif') as f:
        bathymetry_raster = tifffile.imread(f)
# Bathymetry raster is upside down from how we expect it, so flip it
bathymetry_raster = bathymetry_raster[::-1]

In [4]:
# The activate data is at 1/100th of a degree  while the bathymetry is at 1/120th, so
# compromise on 1/10th of a degree since that's easier to plot anyway
h, w = bathymetry_raster.shape
raster = bathymetry_raster.reshape(h // 12, 12, w // 12, 12).mean(axis=(1, 3))
np.savez('../../untracked/bathymetry.npz', bathymetry=raster)
# We multiply presence by 100 here to put this on a similar scale as another presence raster
# I am using. A bit arbitrary, but makes this simpler to keep in sync.
h, w = activity_raster.shape
raster = activity_raster.reshape(h // 10, 10, w // 10, 10).mean(axis=(1, 3))                               
np.savez('../../untracked/presence.npz', presence=100 * raster)